### Importing the Necessary Libraries

In [1]:

import yfinance as yf
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import sklearn
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.stats import norm
import seaborn as sns
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel
import warnings
warnings.filterwarnings("ignore")

### Downloading Financial Data

In [2]:
tickers = pd.read_html('https://ournifty.com/stock-list-in-nse-fo-futures-and-options.html#:~:text=NSE%20F%26O%20Stock%20List%3A%20%20%20%20SL,%20%201000%20%2052%20more%20rows%20')[0]
tickers = tickers.SYMBOL.to_list()
ct = 0


for count in range(len(tickers)):
        tickers[count] = tickers[count] + ".NS"

In [3]:
price_data = yf.download(tickers , start = '2011-01-01', end = '2023-12-31')

[*********************100%***********************]  188 of 188 completed


5 Failed downloads:
['MIDCPNIFTY.NS', 'NIFTY.NS', 'BANKNIFTY.NS', 'FINNIFTY.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')
['L&TFH.NS']: Exception('%ticker%: No data found, symbol may be delisted')


### Verifying the Data

In [4]:



adj_close = price_data['Adj Close']

# adj_close.describe()

adj_close.isna().sum()

adj_close.dropna(axis = 1,how = 'any',inplace = True)

adj_close.isna().sum()


AARTIIND.NS      0
ABB.NS           0
ABBOTINDIA.NS    0
ACC.NS           0
ADANIENT.NS      0
                ..
VEDL.NS          0
VOLTAS.NS        0
WIPRO.NS         0
ZEEL.NS          0
ZYDUSLIFE.NS     0
Length: 147, dtype: int64

In [5]:
adj_close.head()

,AARTIIND.NS,ABB.NS,ABBOTINDIA.NS,ACC.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,APOLLOTYRE.NS,ASHOKLEY.NS,...,TRENT.NS,TVSMOTOR.NS,UBL.NS,ULTRACEMCO.NS,UPL.NS,VEDL.NS,VOLTAS.NS,WIPRO.NS,ZEEL.NS,ZYDUSLIFE.NS
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-03 00:00:00,12.195271,690.747009,1184.816162,868.715149,83.078537,134.131088,112.454544,438.291351,58.281979,23.744942,...,90.173134,61.570709,503.905884,1034.287964,95.685493,113.054672,199.702087,159.698929,130.966904,133.390594
2011-01-04 00:00:00,11.978837,688.916321,1184.816162,867.866089,83.283409,137.367599,113.634567,434.714264,59.939880,23.975651,...,89.269066,60.725552,497.769562,1036.665405,94.468910,111.625290,197.312592,160.227859,129.607849,133.242905
2011-01-05 00:00:00,11.668314,678.059448,1180.696655,860.629639,83.501137,145.736374,110.802536,429.866302,57.134186,23.248035,...,88.070007,58.823906,483.208069,1019.882019,92.460129,113.686279,195.321304,160.558441,127.206856,133.529602
2011-01-06 00:00:00,11.649493,680.571411,1164.217041,826.791382,82.521477,143.285812,105.059830,426.571686,55.986397,23.123810,...,85.905006,55.781300,478.581451,961.211853,91.526482,114.417625,193.285782,160.789932,124.851181,133.607773
2011-01-07 00:00:00,11.197817,665.031372,1150.648438,810.418030,80.434105,136.165451,101.362488,431.984406,54.668583,22.005775,...,84.030258,53.457081,460.708344,939.246155,90.875771,110.312271,189.878510,157.682236,122.767288,133.990005


In [6]:
adj_close.describe()

,AARTIIND.NS,ABB.NS,ABBOTINDIA.NS,ACC.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,APOLLOTYRE.NS,ASHOKLEY.NS,...,TRENT.NS,TVSMOTOR.NS,UBL.NS,ULTRACEMCO.NS,UPL.NS,VEDL.NS,VOLTAS.NS,WIPRO.NS,ZEEL.NS,ZYDUSLIFE.NS
count,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,...,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000,3204.000000
mean,297.063008,1343.283425,8248.813601,1424.921994,553.228411,367.796844,227.635278,1824.156184,171.427587,72.484017,...,483.654869,426.946883,1006.686177,3853.710903,385.214286,103.304601,477.508526,258.120180,292.512627,314.440124
std,286.158754,925.009633,7139.020630,415.044650,942.190431,227.753768,102.571769,1446.205477,87.257561,44.966223,...,531.451633,370.853723,363.994901,2101.565304,229.354311,61.522726,335.349986,133.208073,128.178339,138.212688
min,8.350097,386.908112,1075.281860,759.155457,18.779579,101.367104,92.473106,426.571686,39.407288,9.402225,...,73.086349,26.950359,345.677338,853.190186,62.702538,22.644766,59.632744,107.209503,97.896736,110.519257
25%,28.311460,742.356476,1605.883148,1138.591278,42.801017,173.367840,165.744843,917.552017,97.435783,21.700575,...,112.802507,83.579027,753.764008,1964.969177,124.801828,62.922945,161.142117,173.276230,199.405582,173.372181
50%,207.789986,1096.222168,4783.156982,1341.079834,81.776360,317.625412,193.481323,1234.560425,167.125931,71.268349,...,257.486435,377.536697,933.806580,3717.075928,413.758331,81.043434,451.450317,205.543770,262.130402,324.804016
75%,490.550163,1333.039185,15166.529541,1618.373932,306.934296,405.979774,242.629517,2061.402100,217.414684,108.233685,...,671.668381,579.171494,1326.047272,4518.978638,560.899979,122.197039,693.187149,328.426399,382.206192,401.924538
max,1146.340942,4870.989746,24555.800781,2725.959229,4159.459961,1089.625000,582.585999,5755.308105,462.450012,183.602234,...,3052.855469,2040.681885,1785.150024,10503.049805,818.247742,276.106934,1323.253540,711.108154,588.149231,689.200012


In [7]:
returns = (adj_close - adj_close.shift(1))/adj_close

In [8]:
returns.dropna(axis = 0,inplace = True)

In [9]:
returns.head()

,AARTIIND.NS,ABB.NS,ABBOTINDIA.NS,ACC.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,APOLLOTYRE.NS,ASHOKLEY.NS,...,TRENT.NS,TVSMOTOR.NS,UBL.NS,ULTRACEMCO.NS,UPL.NS,VEDL.NS,VOLTAS.NS,WIPRO.NS,ZEEL.NS,ZYDUSLIFE.NS
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-04 00:00:00,-0.018068,-0.002657,0.000000,-0.000978,0.002460,0.023561,0.010384,-0.008229,0.027659,0.009623,...,-0.010127,-0.013918,-0.012328,0.002293,-0.012878,-0.012805,-0.012110,0.003301,-0.010486,-0.001108
2011-01-05 00:00:00,-0.026613,-0.016012,-0.003489,-0.008408,0.002607,0.057424,-0.025559,-0.011278,-0.049107,-0.031298,...,-0.013615,-0.032328,-0.030135,-0.016456,-0.021726,0.018129,-0.010195,0.002059,-0.018875,0.002147
2011-01-06 00:00:00,-0.001616,0.003691,-0.014155,-0.040927,-0.011872,-0.017103,-0.054661,-0.007723,-0.020501,-0.005372,...,-0.025202,-0.054545,-0.009667,-0.061038,-0.010201,0.006392,-0.010531,0.001440,-0.018868,0.000585
2011-01-07 00:00:00,-0.040336,-0.023367,-0.011792,-0.020204,-0.025951,-0.052292,-0.036476,0.012530,-0.024106,-0.050806,...,-0.022310,-0.043478,-0.038795,-0.023387,-0.007160,-0.037216,-0.017944,-0.019709,-0.016974,0.002853
2011-01-10 00:00:00,-0.033884,-0.043141,0.004224,0.013581,-0.074502,-0.048789,-0.010985,0.031141,-0.055829,-0.031614,...,-0.003523,-0.030130,0.046179,0.007735,-0.030809,-0.032996,-0.035473,-0.026582,-0.012705,-0.002209


### Pair Selection is Based on Kendall's Tau


In [10]:
##############################################
# Computing Stocks with similar Kendall's Tau
##############################################

ticks = returns.columns
ideal = []

print(len(ticks))

for tc in range(len(ticks)):

    for rc in range(tc+1,len(ticks)):

        if(ticks[rc] == ticks[tc]):
            continue

        u1 = st.rankdata(adj_close[ticks[tc]][1:])
        u2 = st.rankdata(adj_close[ticks[rc]][1:])

        tau,p_val = st.kendalltau(u1,u2)
        
        if(tau>=0.8):
            ideal.append([ticks[tc],ticks[rc],tau])

ideal.sort(key = lambda x:x[2])

print((ideal))

147
[['NESTLEIND.NS', 'RELIANCE.NS', 0.8000096724879029], ['DEEPAKNTR.NS', 'INFY.NS', 0.8000743908902405], ['INFY.NS', 'SRF.NS', 0.8000939558846031], ['HAVELLS.NS', 'SRF.NS', 0.8001024980918414], ['KOTAKBANK.NS', 'MRF.NS', 0.8001434879122763], ['MRF.NS', 'NESTLEIND.NS', 0.8002625587844059], ['DABUR.NS', 'TECHM.NS', 0.8002957998917666], ['BAJAJFINSV.NS', 'RAMCOCEM.NS', 0.8003434549576425], ['AARTIIND.NS', 'GODREJCP.NS', 0.8003548855277196], ['CANFINHOME.NS', 'MARICO.NS', 0.8003810616097307], ['GODREJCP.NS', 'JKCEMENT.NS', 0.8004248137366908], ['TCS.NS', 'ULTRACEMCO.NS', 0.8006031620674458], ['NESTLEIND.NS', 'TATACONSUM.NS', 0.8006600492844452], ['HDFCBANK.NS', 'VOLTAS.NS', 0.8009242439796364], ['CHAMBLFERT.NS', 'GODREJPROP.NS', 0.8011575011237909], ['ABBOTINDIA.NS', 'HAVELLS.NS', 0.8012133175360652], ['BAJAJFINSV.NS', 'MPHASIS.NS', 0.8014963562076912], ['IGL.NS', 'SHREECEM.NS', 0.8014967901339081], ['GODREJCP.NS', 'TCS.NS', 0.8015001716755465], ['DEEPAKNTR.NS', 'GODREJCP.NS', 0.80155106

In [11]:
best_pair = ['AARTIIND.NS','ATUL.NS']

#### Using a Rolling Window with Formation_Period as 126 days and Trading Period as 63 days

In [12]:
formation_period = 126
trading_period = 63

In [13]:
stock_1 = (returns['AARTIIND.NS'])
stock_2 = (returns['ATUL.NS'])

stocks_2D = np.transpose(np.vstack((stock_1,stock_2)))
print(stocks_2D)

[[-0.01806799  0.00763764]
 [-0.0266125  -0.00131851]
 [-0.00161559 -0.01336197]
 ...
 [ 0.01958966 -0.0007899 ]
 [-0.00953643  0.0060829 ]
 [ 0.01531704  0.01201269]]


In [14]:
stocks_2D[:,0] = (stocks_2D[:,0] - stocks_2D[:,0].mean())/stocks_2D[:,0].std()

In [15]:
stocks_2D[:,1] = (stocks_2D[:,1] - stocks_2D[:,1].mean())/stocks_2D[:,1].std()

In [16]:
print(adj_close['AARTIIND.NS'][2])

res = []

11.668313980102539


### Function to Generate the Cumulative Profit based on the signals

In [17]:
def compute_discrete(signal_u,signal_v):

    invest_u = 100
    invest_v = 100

    cumulative_invest_u = 0

    cumulative_invest_v = 0

    profit_realized_u = 0

    profit_realized_v = 0

    

    for i in range(0,(signal_u.shape[1])):

        if(signal_u[0][i] == 1):

            cumulative_invest_u += invest_u

            price_paid = adj_close['AARTIIND.NS'][i]

            stocks_purchased = invest_u/price_paid # invest/price

            while(signal_u[0][i] == 1 and i < signal_u.shape[1] - 1):

                i+=1
                

            price_sold = adj_close['AARTIIND.NS'][i]

            profit_realized_u += (price_sold - price_paid)*(stocks_purchased)


        elif(signal_u[0][i] == -1):

            cumulative_invest_u += invest_u

            price_sold = adj_close['AARTIIND.NS'][i]

            stocks_sold = (invest_u/price_sold)

            while(signal_u[0][i] == -1 and i < (signal_u.shape[1])-1):
                
                i+=1
                
            # print(i)
            price_paid = adj_close['AARTIIND.NS'][i]

            # print(price_sold,invest_u,stocks_sold,price_paid)

            profit_realized_u += (price_sold - price_paid)*(stocks_sold)

    
    for i in range(0,(signal_v.shape[1])):

        if(signal_v[0][i] == 1):

            # print("Hello --> v = 1")

            cumulative_invest_v += invest_v

            price_paid = adj_close['ATUL.NS'][i]

            stocks_purchased = (invest_v/price_paid)

            while(signal_v[0][i] == 1 and i < signal_u.shape[1] - 1):

                i+=1 

            price_sold = adj_close['ATUL.NS'][i]

            profit_realized_v += (price_sold - price_paid)*(stocks_purchased)

        elif(signal_v[0][i] == -1):

            cumulative_invest_v += invest_v

            price_sold = adj_close['ATUL.NS'][i]

            stocks_sold = (invest_u/price_sold)

            while(signal_v[0][i] == -1 and i < signal_u.shape[1] - 1):

                i+=1
                

            price_paid = adj_close['ATUL.NS'][i]

 

            profit_realized_v += (price_sold - price_paid)*(stocks_sold)

    if(cumulative_invest_u == 0 and cumulative_invest_v == 0):

        print("Total Investment in this Trading Period ============> ",(cumulative_invest_v + cumulative_invest_u))

        return

    print("Total Investment in this Trading Period ============> ",(cumulative_invest_v + cumulative_invest_u))

    print("Total Profit/Loss Realized in this Trading Period ==========> ",(profit_realized_u + profit_realized_v))

    res.append(profit_realized_u + profit_realized_v)

    print("Percentage terms =======> ",((profit_realized_u + profit_realized_v)/(cumulative_invest_u + cumulative_invest_v)) + 1)



    







### Function to Generate the Signals

In [18]:
def signal_generation(p_u,p_v):


    signal_u = np.zeros((1,len(p_u)))

    signal_v = np.zeros((1,len(p_v)))

    # print(signal_u.shape)

    # print(p_u)

    # print(p_v)

    for i in range(0,len(p_u)):

        if(p_u[i] >= 0.7 and p_v[i] <= 0.3):

            signal_u[0][i] = -1
            signal_v[0][i] = 1


            while(p_v[i] < 0.5 and p_u[i] > 0.5 and i < len(p_u)-1):

                signal_u[0][i] = -1
                signal_v[0][i] = 1
                i+=1

        elif(p_u[i] <= 0.3 and p_v[i] >= 0.7 ):

            signal_u[0][i] = 1
            signal_v[0][i] = -1

            while(p_v[i] > 0.5 and p_u[i] < 0.5 and i < len(p_u)-1):

                signal_u[0][i] = -1
                signal_v[0][i] = 1
                i+=1

    # print(signal_u)
    # print(signal_v)
    compute_discrete((signal_u),(signal_v))



### Implementing the Gaussian Process Regression with combination of Various Kernels

In [19]:
##########################################
# Using Simple RBF Kernel and lookback = 1
##########################################



def Rolling_Window(stocks_2D):

    positions_per_window = []

    for i in range(0,len(returns) - (trading_period + formation_period),trading_period):

        X = stocks_2D[i : i + formation_period , :]

        Y = stocks_2D[i + 1 : i + formation_period + 1 , :]

        kernel = 63*RBF(length_scale=1) + 0.063**2*RationalQuadratic(length_scale=1.2, alpha=0.78) + 0.18**2 * RBF(length_scale=0.134) + WhiteKernel(noise_level=0.19**2)

        gaussian_process = GaussianProcessRegressor(kernel=kernel,alpha = 1e-10,n_restarts_optimizer=10,normalize_y=False)

        # Y -= Y.mean()
        
        gaussian_process.fit(X, Y)

        kernel_upd = gaussian_process.kernel_

        probabilities = []

        Prob_U = []

        Prob_V = []

        positions = 0

        for j in range(i+formation_period, i+formation_period+trading_period):

            kernel_matrix = kernel_upd(stocks_2D[j - 1].reshape(-1,1))

            bivariate_mean_U = (kernel_matrix[0][1]) * (1/kernel_matrix[0][0]) * (stocks_2D[j-1][1])

            bivariate_mean_V = (kernel_matrix[0][1]) * (1/kernel_matrix[0][0]) * (stocks_2D[j-1][0])

            bivariate_std_dev = kernel_matrix[0][0] - ((kernel_matrix[0][1]) * (1/kernel_matrix[1][1]) * (kernel_matrix[1][0]))

            conditional_probability_U = norm.cdf((stocks_2D[j,0] - bivariate_mean_U)/bivariate_std_dev)

            conditional_probability_V = norm.cdf((stocks_2D[j,1] - bivariate_mean_V)/bivariate_std_dev)

            Prob_U.append(np.round(conditional_probability_U,3))

            Prob_V.append(np.round(conditional_probability_V,3))

            if((conditional_probability_U >= 0.7 and conditional_probability_V <= 0.3) or (conditional_probability_U <= 0.3 and conditional_probability_V >= 0.7)):

                positions = positions + 1

            probabilities.append([conditional_probability_U , conditional_probability_V])

        probabilities = np.array(probabilities)

        signal_generation(np.asarray(Prob_U),np.asarray(Prob_V))

        # print(probabilities)

        # plt.plot(probabilities[0,:])

        positions_per_window.append(positions)

        dataframe_columns = ['Conditional_Probability_U','Conditional_Probability_V']

        probabilities_df = pd.DataFrame(probabilities,columns=dataframe_columns)

        # print(probabilities_df)

        # plt.figure(figsize=(18,4))

        # plt.plot(probabilities_df['Conditional_Probability_U'])

        # plt.plot(probabilities_df['Conditional_Probability_V'])

        # plt.legend(['Conditional_Probability_U','Conditional_Probability_V'])

        # plt.tight_layout()

        # plt.show()


    positions_per_window = np.array(positions_per_window)

    positions_df = pd.DataFrame(positions_per_window)

    print(positions_df)



In [20]:
##############
# With Returns
##############

Rolling_Window(stocks_2D)
print(sum(res))

Total Investment in this Trading Period ============>  1400
Total Profit/Loss Realized in this Trading Period ==========>  7.349656129335329
Percentage terms =======>  1.0052497543780967
Total Investment in this Trading Period ============>  400
Total Profit/Loss Realized in this Trading Period ==========>  0.46186433935947946
Percentage terms =======>  1.0011546608483988
Total Investment in this Trading Period ============>  800
Total Profit/Loss Realized in this Trading Period ==========>  1.6420705055483495
Percentage terms =======>  1.0020525881319355
Total Investment in this Trading Period ============>  1800
Total Profit/Loss Realized in this Trading Period ==========>  4.146284238264229
Percentage terms =======>  1.00230349124348
Total Investment in this Trading Period ============>  2600
Total Profit/Loss Realized in this Trading Period ==========>  12.08436241567894
Percentage terms =======>  1.004647831698338
Total Investment in this Trading Period ============>  600
Total Pr

In [21]:
print("The Total Profit Earned ======> ")
print(sum(res))

The Total Profit Earned ======> 
116.3678123487423
